In [3]:
import numpy as np
import numpy.linalg as la
import time

def min_jerk(a, t, step_time):

    ini_pos_1= a[0]
    end_pos_1= a[1]
    ini_pos_2= a[2]
    end_pos_2= a[3]
    D= a[4]
    

    pos_1_desire=  ini_pos_1+ (end_pos_1- ini_pos_1)*((10* (t/D)**3)-\
        15* (t/D)**4+ 6*(t/D)**5)

    pos_2_desire=  ini_pos_2+ (end_pos_2- ini_pos_2)*((10* (t/D)**3)-\
        15* (t/D)**4+ 6*(t/D)**5)

    vel_1_desire= 1.0/D *(end_pos_1- ini_pos_1)* (30*(t/D)**2-\
        60*(t/D)**3 + 30*(t/D)**4)

    vel_2_desire= 1.0/D *(end_pos_2- ini_pos_2)* (30*(t/D)**2-\
        60*(t/D)**3 + 30*(t/D)**4)

    vel_1= ((ini_pos_1+ (end_pos_1- ini_pos_1)*((10* ((t+step_time)/D)**3)-\
        15* ((t+step_time)/D)**4+ 6*((t+step_time)/D)**5))-(ini_pos_1+\
        (end_pos_1- ini_pos_1)*((10* (t/D)**3)-15* (t/D)**4+ 6*(t/D)**5)))/step_time

    vel_2= ((ini_pos_2+ (end_pos_2- ini_pos_2)*((10* ((t+step_time)/D)**3)-\
        15* ((t+step_time)/D)**4+ 6*((t+step_time)/D)**5))-(ini_pos_2+\
        (end_pos_2- ini_pos_2)*((10* (t/D)**3)-15* (t/D)**4+ 6*(t/D)**5)))/step_time

    return pos_1_desire, pos_2_desire,vel_1_desire, vel_2_desire, vel_1, vel_2

def pos_error(a,pos_1_desire, pos_2_desire):

    q=np.array([a[0], a[2]])
    qd=np.array([pos_1_desire,pos_2_desire])
    q_mat= np.mat(q).T
    qd_mat=np.mat(qd).T
    pos_err= qd_mat-q_mat
    #print(pos_err)

    return pos_err

def vel_error(a, vel_1_desire, vel_2_desire, vel_1, vel_2 ):

    v=np.array([vel_1, vel_2])
    vd=np.array([vel_1_desire,vel_2_desire])
    v_mat= np.mat(v).T
    vd_mat=np.mat(vd).T
    vel_err= vd_mat-v_mat
    #print(vel_err)

    return vel_err

def dynamic_oiac(action):
    a=action
    vel_err=vel_error(a, vel_1_desire, vel_2_desire, vel_1, vel_2)
    pos_err=pos_error(a,pos_1_desire, pos_2_desire)
    #print("dyn:",vel_err,pos_err)
    track_err= pos_err + beta* vel_err
    adapt_scale= a_const/ (1+ c_const* la.norm(track_err)*la.norm(track_err))
    #print("track:",track_err,"adapt:",adapt_scale)
    k_mat= track_err*pos_err.T/adapt_scale
    b_mat= track_err*vel_err.T/adapt_scale
 
    return k_mat, b_mat


def imp_control(a,k_mat,b_mat):
    

       
    vel_err=vel_error(a, vel_1_desire, vel_2_desire, vel_1, vel_2)
    pos_err=pos_error(a,pos_1_desire, pos_2_desire)

    torque= k_mat* pos_err+ b_mat* vel_err
    torque=torque.flatten()
    torque_1=torque[0,0]
    torque_2=torque[0,1]
    
    return torque_1, torque_2


start_loop_time=time.time()
t_step=1e-8
sum_step=1e-8
# k_mat= np.mat(np.zeros((2,2)))
# b_mat= np.mat(np.zeros((2,2)))
action=np.array([-0.5*np.pi, 0.5*np.pi, -0.3*np.pi, 0.6*np.pi, 0.1])
a_const = 0.2
c_const = 5.0
beta = 0.05


while(sum_step<=action[4]):
    pos_1_desire, pos_2_desire,vel_1_desire, vel_2_desire,vel_1, vel_2= min_jerk(action,sum_step,t_step)
    k_mat, b_mat=dynamic_oiac(action)
    tor1,tor2=imp_control(action,k_mat, b_mat )
    print(k_mat,"b",b_mat)
    end_loop_time=time.time()
    t_step= end_loop_time - start_loop_time
    sum_step = sum_step + t_step
    




[[0. 0.]
 [0. 0.]] b [[2.22066010e-23 1.99859409e-23]
 [1.99859409e-23 1.79873468e-23]]
[[-3.11773211e-07 -2.80595890e-07]
 [-2.80595890e-07 -2.52536301e-07]] b [[0.00152206 0.00136985]
 [0.00136985 0.00123287]]
[[-1.62083295e-05 -1.45874966e-05]
 [-1.45874966e-05 -1.31287469e-05]] b [[0.01665189 0.0149867 ]
 [0.0149867  0.01348803]]
[[-0.00019837 -0.00017853]
 [-0.00017853 -0.00016068]] b [[0.07726984 0.06954286]
 [0.06954286 0.06258857]]
[[-0.00116825 -0.00105142]
 [-0.00105142 -0.00094628]] b [[0.21795086 0.19615577]
 [0.19615577 0.1765402 ]]
[[-0.00462188 -0.00415969]
 [-0.00415969 -0.00374372]] b [[0.47739643 0.42965679]
 [0.42965679 0.38669111]]
[[-0.01194672 -0.01075205]
 [-0.01075205 -0.00967684]] b [[0.72473274 0.65225946]
 [0.65225946 0.58703352]]
[[-0.02064322 -0.0185789 ]
 [-0.0185789  -0.01672101]] b [[0.78070498 0.70263448]
 [0.70263448 0.63237104]]
[[-0.0167786  -0.01510074]
 [-0.01510074 -0.01359067]] b [[0.41204415 0.37083973]
 [0.37083973 0.33375576]]
[[0.03860643 0.0